In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
from tqdm import trange
import multiprocessing as mp
import seaborn as sns

from cvx.stat_arb.ccp import *
from cvx.stat_arb.utils import *
from cvx.stat_arb.backtest import *

sns.set()

%load_ext autoreload
%autoreload 2

In [2]:
naics_to_sector = {
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities', 
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44': 'Retail Trade',
    '45': 'Retail Trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}

In [3]:
data = pd.read_csv("../data/asset_data_2010to2023_full.csv")
prices = pd.read_csv("../data/prices_500_cleaned.csv", index_col=0, header=0, parse_dates=True).dropna(axis=1, how='any')
data = data[data['PERMCO'].isin(prices.columns.astype(int))]

/var/folders/mr/mp0qg4bx6sj56pg6fs3s65kc0000gr/T/ipykernel_5148/2449403297.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/asset_data_2010to2023_full.csv")


In [4]:
NAICS = pd.pivot_table(data, values='NAICS', index='date', columns='PERMCO',\
     fill_value=None)
NAICS = NAICS.fillna(method='ffill')
NAICS = NAICS.fillna(method='bfill')
NAICS = NAICS.applymap(lambda x: str(x)[:2]).astype(int)

# Get dict of sectors
asset_to_sector = {}
sector_to_asset = {}
assets_visited = []
for asset in NAICS.columns:
    if asset in assets_visited:
        print("fdsj")
    assets_visited.append(asset)
    if asset in prices.columns.astype(int): # TODO: They have different columns...
        sectors_temp = []
        for n in NAICS[asset].unique():
            try:
                sector = naics_to_sector[str(n)]

                if sector not in sector_to_asset:
                    sector_to_asset[sector] = [asset]
                else:
                    if asset not in sector_to_asset[sector]:
                        sector_to_asset[sector].append(asset)
                sectors_temp.append(sector)

            except KeyError:
                print("Sector " + str(n) + " does not exist...")
        asset_to_sector[asset] = sectors_temp

In [5]:
# Sace sector_to_asset and asset_to_sector to file
import pickle
with open('../data/sector_to_asset.pkl', 'wb') as f:
    pickle.dump(sector_to_asset, f)
with open('../data/asset_to_sector.pkl', 'wb') as f:
    pickle.dump(asset_to_sector, f)

In [6]:
for sect, assets in sector_to_asset.items():
    print("\n", sect)
    print(len(assets))


 Manufacturing
203

 Management of Companies and Enterprises
17

 Finance and Insurance
96

 Professional, Scientific, and Technical Services
26

 Transportation and Warehousing
30

 Retail Trade
42

 Real Estate and Rental and Leasing
32

 Wholesale Trade
34

 Mining, Quarrying, and Oil and Gas Extraction
36

 Information
61

 Administrative and Support and Waste Management and Remediation Services
20

 Accommodation and Food Services
11

 Arts, Entertainment, and Recreation
7

 Utilities
28

 Construction
4

 Agriculture, Forestry, Fishing and Hunting
1

 Health Care and Social Assistance
3
